### Explore the training set
There are 433 variables in training set in total: 383 numeric columns and 50 non-numeric columns.<br>
The description of the variables are in 'Variable Description.txt' file.<br>
In this file, we explore the features' correlation with the target and their multicollinearity.

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# Make better use of Jupyter Notebook cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

Import the raw training sets

In [2]:
X_train, y_train = pd.read_pickle('data/X_train.pkl'), pd.read_pickle('data/y_train.pkl')
df = pd.concat([X_train, y_train], axis = 1)

Check correlation between the features and the target

In [3]:
cor = df.corrwith(y_train)
cor = cor[abs(cor).sort_values(ascending=False).index.values]
cor = cor[1:]

The variables that have higher than 10% correlation with the target are all Vesta engineered rich features which includes counting, ranking, etc. V257 may have to do with counts of payments within a time-period or place

In [4]:
cor_vars = cor[abs(cor) > .1].index.values
round(pd.DataFrame(cor[abs(cor) > .1]).T,2)

,V257,V246,V244,V242,V201,V200,V189,V258,V188,V45,V158,V156,V228,V149,V44,V87,V86,V170,V52,V147,V230,V199,V157,V155,V148,V51,V171,V243,V40,V190,V39,V154,V38,V43,V79,V146,V140,V42,V94,V74,V33,V17,V18,V81,V93,V92,V34,V153,V50,V80,V247,V15,V37,V16,V73,V58,V85,V84,V57,V21,V176,V222,V47,V31,V72,V32,V77,V22,V71,V262,V63,V78,V252,V60,V59,V197,V64,V186,V23,V259,V194,V139,V195,D8,V239,V229,V123,V198,V302,V304,V221,V260,V46,D7,V303,V24,V111,id_01,V184,V238,V249,V48,V113,V283,V125,V49,V90,V29,V261,V185,V91,V69,V112,V30,V282,V70
0,0.38,0.37,0.37,0.36,0.33,0.32,0.31,0.3,0.3,0.28,0.28,0.27,0.27,0.27,0.26,0.25,0.25,0.25,0.24,0.24,0.24,0.23,0.23,0.23,0.23,0.22,0.22,0.22,0.21,0.21,0.2,0.2,0.2,0.2,0.2,0.19,0.19,0.19,0.19,0.19,0.18,0.18,0.18,0.18,0.18,0.18,0.18,0.18,0.18,0.18,0.18,0.18,0.18,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.17,0.16,0.16,0.16,0.16,0.16,0.16,0.16,0.16,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.15,0.14,-0.14,0.14,0.14,0.14,0.14,0.13,0.13,0.13,0.13,0.13,-0.13,0.12,0.12,0.12,-0.12,0.12,0.12,0.12,-0.12,0.11,0.11,0.11,-0.11,-0.11,-0.11,0.11,0.11,-0.11,-0.1,0.1,-0.1,0.1,-0.1


#### Multicollinearity check
VIF could not be implemented because of massive number of missing values

Read the saved correlations among features

In [5]:
x_cor = pd.read_pickle('EDA/x_cor_list.pkl')

In [6]:
# c = df.corr().abs()
# x_cor = c.unstack().sort_values(kind="quicksort")
# x_cor = pd.DataFrame(x_cor).reset_index(level=[0,1])
# x_cor.columns = ['var1','var2','correlation']
# x_cor = x_cor.loc[x_cor.var1 != x_cor.var2].sort_values(by='correlation', ascending = False).reset_index(drop=True)

74\% of the variables have high correlation with other variables. There's severe multicollinearity.

In [7]:
round(len(np.unique(x_cor.loc[x_cor.correlation >= 0.8,['var1']].values)) / len(X_train.columns),4)

0.7413

**Multicollinearity among Vesta-engineered features**<br>
75% of the Vesta-engineered features have correlation with one another greater than 0.8 and these multicollinear features account for 59% of the total features.

In [8]:
ratio = len(np.unique(multicor_x.loc[multicor_x.var1.map(lambda x: x[0] == 'V'),'var1'])) / \
sum(X_train.columns.map(lambda x: x[0] == 'V'))
round(ratio, 2)

0.75

In [9]:
ratio = len(np.unique(multicor_x.loc[multicor_x.var1.map(lambda x: x[0] == 'V'),'var1'])) / \
len(X_train.columns)
round(ratio, 2)

0.59

Highly correlated pairs among Vesta-engineered features<br>
*For logistic regression, multicollinearity could cause variance inflation in coefficients which are unstable across different samples*

In [10]:
multicor_x.loc[mask,:].T

,2,4,6,8,10,12,14,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,68,70,72,76,78,80,82,84,86,88,90,92,94,96,98,100,102,106,108,110,112,114,116,118,120,122,124,126,128,130,134,136,138,140,142,144,146,148,150,152,154,156,158,160,162,164,166,168,170,172,174,176,178,180,182,184,186,188,190,192,194,196,198,200,202,204,206,208,210,212,214,216,218,220,222,224,226,228,230,232,234,236,238,240,242,244,246,250,252,254,256,258,260,262,264,266,268,270,272,274,276,278,280,282,284,286,288,290,292,294,296,298,300,302,304,306,308,310,312,314,316,318,320,322,324,326,328,330,332,334,336,338,340,342,344,346,348,350,352,354,356,358,360,362,364,366,368,370,372,374,376,378,382,384,386,388,390,392,394,396,398,400,402,404,406,408,410,412,414,416,418,420,422,424,426,428,430,432,434,436,438,440,442,444,446,448,450,452,454,456,458,460,462,464,466,468,472,474,476,478,480,482,484,486,488,490,492,494,496,498,500,502,504,506,508,514,516,518,520,...,4652,4654,4656,4658,4660,4662,4664,4666,4668,4670,4672,4674,4676,4678,4680,4682,4684,4686,4688,4690,4692,4694,4696,4698,4700,4702,4704,4706,4708,4710,4712,4714,4716,4718,4720,4722,4724,4726,4728,4730,4732,4734,4736,4738,4740,4742,4744,4746,4748,4750,4752,4754,4756,4758,4760,4762,4764,4766,4768,4770,4772,4774,4776,4778,4780,4782,4784,4786,4788,4790,4792,4794,4796,4798,4800,4802,4804,4806,4808,4810,4812,4814,4816,4818,4820,4822,4824,4826,4828,4830,4832,4834,4836,4838,4840,4842,4844,4846,4848,4850,4852,4854,4856,4858,4860,4862,4864,4866,4868,4870,4872,4874,4876,4878,4880,4882,4884,4886,4888,4890,4892,4894,4896,4900,4902,4904,4906,4908,4910,4912,4914,4916,4918,4920,4922,4924,4926,4928,4930,4932,4934,4936,4938,4940,4942,4944,4946,4948,4950,4952,4954,4956,4958,4960,4962,4964,4966,4968,4970,4972,4974,4976,4980,4982,4984,4986,4988,4990,4992,4994,4996,4998,5000,5002,5004,5006,5008,5010,5012,5014,5016,5018,5020,5022,5024,5026,5028,5030,5032,5034,5036,5038,5040,5042,5044,5046,5048,5050,5052,5054,5056,5058,5060,5062,5064,5066,5068,5070,5072,5074,5076,5078,5080,5082,5084,5086,5088,5090,5092,5094,5096,5098,5100,5102,5104,5106,5108,5110,5112,5114,5116,5118,5120,5122,5124,5126,5128,5130,5132,5134,5136,5138,5140,5142,5144,5146,5148,5150,5152,5154
var1,V322,V323,V97,V322,V95,V279,V279,V280,V177,V322,V103,V105,V101,V323,V97,V324,V279,V295,V294,V293,V101,V177,V330,V132,V132,V167,V133,V322,V134,V102,V323,V127,V318,V164,V332,V269,V333,V213,V217,V332,V168,V103,V332,V133,V101,V329,V168,V95,V126,V308,V323,V96,V332,V233,V132,V316,V332,V323,V316,V132,V97,V316,V323,V101,V323,V211,V323,V127,V177,V177,V295,V95,V333,V306,V132,V322,V167,V177,V299,V177,V333,V167,V101,V316,V134,V95,V179,V279,V164,V323,V323,V323,V213,V333,V96,V132,V167,V318,V133,V126,V133,V167,V276,V164,V164,V298,V322,V164,V211,V293,V233,V317,V164,V332,V17,V103,V279,V329,V318,V134,V318,V322,V211,V133,V133,V178,V324,V180,V232,V97,V97,V103,V103,V308,V333,V202,V324,V333,V101,V179,V331,V293,V217,V95,V143,V324,V179,V316,V332,V143,V102,V324,V318,V323,V280,V306,V178,V332,V308,V324,V96,V128,V127,V233,V202,V179,V326,V97,V132,V103,V280,V164,V103,V96,V132,V307,V179,V332,V143,V127,V212,V332,V331,V202,V179,V295,V328,V306,V179,V177,V164,V153,V126,V102,V97,V100,V179,V179,V133,V16,V216,V331,V331,V127,V164,V127,V318,V126,V31,V280,V126,V126,V324,V324,V317,V97,V179,V331,V128,V126,V322,V323,V322,V134,V95,V96,V133,V279,V331,V323,V323,V179,V213,V103,V212,V331,V132,V127,V266,V324,V179,V324,V177,V250,V128,V134,V133,V256,V217,V133,...,V230,V59,V165,V85,V242,V202,V183,V51,V24,V104,V16,V180,V229,V84,V180,V143,V96,V186,V333,V207,V298,V167,V298,V183,V232,V164,V57,V295,V183,V103,V299,V218,V101,V211,V327,V167,V290,V327,V180,V72,V295,V132,V219,V274,V212,V295,V186,V306,V297,V266,V105,V165,V175,V335,V165,V212,V332,V237,V126,V264,V52,V21,V63,V64,V330,V43,V316,V297,V331,V64,V110,V180,V64,V180,V143,V80,V125,V16,V332,V329,V202,V185,V298,V296,V237,V296,V333,V203,V60,V302,V105,V130,V84,V93,V186,V333,V59,V79,V143,V93,V200,V202,V225,V203,V127,V92,V182,V296,V63,V307,V133,V219,V18,V183,V237,V85,V273,V

Multicollinear features that are not Vesta-engineered features:
- C1-C14 variables are multicollinear with each other 
- D1-D7, D12 variables are multicollinear with each other
- addr2 have collinearity with V15, V58, V79
- Collinearity between TransactionDT and TransactionID should be ignored since TransactionID is an identifier not a numeric value.

In [11]:
multicor_x = x_cor.loc[x_cor.correlation > 0.8][::2]

mask = multicor_x.var1.map(lambda x: x[0] == 'V') & multicor_x.var2.map(lambda x: x[0] == 'V')
multicor_x.loc[~mask,:].T

,0,16,66,74,104,132,248,380,470,510,512,534,536,566,604,700,738,766,770,802,820,854,858,922,930,934,950,988,990,1022,1076,1116,1442,1486,1674,1974,2036,2116,2222,2358,2378,2424,2618,2844,2866,3242,3774,3788,3792,3946,4898,4978
var1,D4,C7,C8,C1,C2,C2,C6,D7,C7,C10,C14,C12,C7,C1,D2,D6,C2,C2,C11,C2,C10,C8,C4,C11,C11,C6,C4,D4,C10,C10,C10,C14,C2,C2,C2,C1,C1,C9,C8,C11,C11,C10,C14,C4,C4,C14,C7,C14,C12,C14,D7,C13
var2,D12,C12,C10,C11,C1,C11,C11,D5,C10,C12,C6,C8,C8,C6,D1,D12,C8,C6,C4,C4,C2,C1,C1,C8,C14,C4,C8,D6,C1,C11,C4,C1,C12,C7,C14,C12,C7,C5,C6,C12,C7,C6,C4,C7,C12,C13,C6,C8,C6,C10,D3,C6
correlation,0.999999,0.999485,0.996975,0.996555,0.995129,0.993992,0.991209,0.98768,0.98548,0.984224,0.984158,0.983565,0.983522,0.982486,0.981351,0.978376,0.976207,0.975186,0.974693,0.97253,0.971044,0.96813,0.968095,0.96328,0.962525,0.962298,0.961136,0.958824,0.958691,0.956718,0.95378,0.951989,0.941808,0.940418,0.936482,0.929644,0.92797,0.925879,0.923104,0.917768,0.917281,0.915741,0.907501,0.898235,0.897756,0.881697,0.861676,0.861391,0.861268,0.854336,0.814747,0.809412


*Finding multicollinear columns that have some correlation(0.1+) with the target*

In [12]:
mask = x_cor.var1.map(lambda x: x in cor_vars) & x_cor.var2.map(lambda x: x in cor_vars)
highcor_colnames = np.unique(x_cor.loc[mask,['var1']].values.squeeze())
highcor_colnames[:10]

array(['D7', 'D8', 'V111', 'V112', 'V113', 'V123', 'V125', 'V139', 'V140',
       'V146'], dtype=object)

*Class imbalance*

NotFraud: Fraud ratio is 27.6 : 1 ==> class weights can be {0:1, 1:27}

In [13]:
round(sum(y_train == 0) / sum(y_train), 1)

27.6